<a href="https://colab.research.google.com/github/Keerthu8999/GoogleColab/blob/main/Copy_of_CrimeNeR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crime Attribute Extraction App

##  Part  1

Loading Dataseet

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip
!ls

In [19]:
import sys
sys.path.append("/content")
from CoNLL2Spacy import *

In [ ]:
# Test File
file = open("/content/Crimetest.txt", "r",encoding = "utf-8")
valList = []
for line in file:
    valList.append(line[:-1])
valList[:15]

In [22]:
TEST_DATA = conll2spacy(valList)

In [ ]:
# Train File
file = open("/content/Crimetrain.txt", "r",encoding = "utf-8")
trainList = []
for line in file:
    trainList.append(line[:-1])
trainList[:15]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [24]:
TRAIN_DATA = conll2spacy(trainList)

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
import spacy
import numpy
import pickle


In [27]:
lang="en"
vectors_loc = "/content/wiki-news-300d-1M.vec"
nlp = spacy.blank(lang)
with open(vectors_loc, "rb") as file_:
    header = file_.readline()
    nr_row, nr_dim = header.split()
    nlp.vocab.reset_vectors(width=int(nr_dim))
    for line in file_:
        line = line.rstrip().decode("utf8")
        pieces = line.rsplit(" ", int(nr_dim))
        word = pieces[0]
        vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")
        nlp.vocab.set_vector(word, vector)  # add the vectors to the vocab

In [29]:
import numpy as np
import spacy

# Define the path to the full word vector file
full_vectors_loc = "/content/wiki-news-300d-1M.vec"

# Define the path to the subset file
subset_vectors_loc = "/content/subset_vectors.vec"

# Define the size of the subset (e.g., 100,000)
subset_size = 100000

# Create a subset of word vectors
def create_subset(full_file, subset_file, subset_size):
    with open(full_file, "rb") as full_file:
        with open(subset_file, "wb") as subset_file:
            # Write the header
            header = full_file.readline()
            subset_file.write(header)

            # Write a subset of vectors
            for _ in range(subset_size):
                line = full_file.readline()
                subset_file.write(line)

# Create the subset file
create_subset(full_vectors_loc, subset_vectors_loc, subset_size)

# Load the subset of word vectors into SpaCy
lang = "en"
nlp = spacy.blank(lang)
with open(subset_vectors_loc, "rb") as file_:
    header = file_.readline()
    nr_row, nr_dim = header.split()
    nlp.vocab.reset_vectors(width=int(nr_dim))
    for line in file_:
        line = line.rstrip().decode("utf8")
        pieces = line.rsplit(" ", int(nr_dim))
        word = pieces[0]
        vector = np.asarray([float(v) for v in pieces[1:]], dtype="f")
        nlp.vocab.set_vector(word, vector)

# Now nlp object contains the subset of word vectors


In [30]:
text = "I lost his doctor in patient yesterday"
doc = nlp(text)
print("similarity btw",doc[3] , "and", doc[5]," :-", doc[3].similarity(doc[5]))

similarity btw doctor and patient  :- 0.6411671042442322


# Part  2

Steps involved :-
1. Loading required library
2. Setting up the parameters for traning
3. Traning and saving the best model

In [32]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing
from spacy.util import minibatch, compounding


# For evaluateing the model from testing set
from spacy.training import *
from spacy.scorer import Scorer

from tqdm import tqdm


In [31]:
def train_spacy(TRAIN_DATA,TEST_DATA,iterations,droprate = 0.5,modelName = "modelTrained"):

    lang="en"
    vectors_loc = "/content/subset_vectors.vec"
    modiner = spacy.blank(lang)
    with open(vectors_loc, "rb") as file_:
        header = file_.readline()
        nr_row, nr_dim = header.split()
        modiner.vocab.reset_vectors(width=int(nr_dim))
        for line in file_:
            line = line.rstrip().decode("utf8")
            pieces = line.rsplit(" ", int(nr_dim))
            word = pieces[0]
            vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")
            modiner.vocab.set_vector(word, vector)  # add the vectors to the vocab
    ner = nlp.create_pipe("ner")
    if 'ner' not in modiner.pipe_names:
        modiner.add_pipe("ner", last=True)

    # setting up f1score
    f1score = 0.0000


    # add labels that will be involved in training
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])


    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in modiner.pipe_names if pipe != 'ner']
    with modiner.disable_pipes(*other_pipes):  # only train NER
        optimizer = modiner.begin_training()

        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}



            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(2.0, 16.0, 1.01))
            size_of_batches = sys.getsizeof(batches)
            print("Size of batches:", size_of_batches, "bytes")
            examples = []
            for batch in batches:
              for text, annotation in tqdm(batch):
                  example = (Example.from_dict(nlp.make_doc(text), annotation))
                  examples.append(example)
                  modiner.update(
                        [example],  # batch of annotations
                        drop=droprate,  # dropout - make it harder to memorise data
                        losses=losses,
                    )
                  print(losses)


            # Evaluating the Current Model Score on test data
            results = evaluate(modiner, TEST_DATA)
            print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])



            # loading previous best saved model in start of traning
            if f1score == 0.00:
                try:
                    pnlp = spacy.load(modelName)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")

            print("Best Sccore :- ",f1score)
            print("------------------------------------")
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]

                # Save our trained Model if the score if grater than best score else no change in previous model
                modiner.to_disk(modelName)

    print("-----Best Model is Saved-----")



In [33]:
def evaluate(ner_model, examples):
    scorer = Scorer()

    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)

        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)

        # Evaluating the tags
        gold = Example(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)


    return scorer.scores


In [34]:
def loadNERModel(modelName = "modelTrained"):
    nlp = spacy.load(modelName)
    return nlp

In [35]:
def score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [ ]:
# traning the model with 100 iterations


train_spacy(TRAIN_DATA,TEST_DATA, 1,droprate = 0.55, modelName = "CrimeNER")

Starting iteration 0
Size of batches: 104 bytes


100%|██████████| 2/2 [00:00<00:00, 22.60it/s]


{'ner': 7.199999511241913}
{'ner': 29.82206642627716}


100%|██████████| 2/2 [00:00<00:00, 34.97it/s]


{'ner': 47.45398283004761}
{'ner': 61.28091657161713}


...

{'ner': 16437.259696543846}


  0%|          | 0/16 [00:00<?, ?it/s]

{'ner': 16437.75403022472}


 19%|█▉        | 3/16 [00:00<00:00, 24.61it/s]

{'ner': 16440.523541786584}
{'ner': 16442.157612044233}
{'ner': 16448.207122078675}
{'ner': 16448.605236407562}


In [ ]:
# loading the saved model
pnlp = loadNERModel("CrimeNER")

In [ ]:
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 86.47551444580847
Precision of Model is :- 87.0393931979447
Recall of Model is :- 85.91889476607976


In [ ]:
from spacy import displacy


In [ ]:
testcase = pnlp('he was killed due to harassment')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('A woman was assault by two men near Shahdara')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('My name is Pooja I was near Domino-pizza in Pinjore a man on bike snatched my purse')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('My name is Amit I saw a woman killing a dog on 15th july near Kalujhanda')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('A car hit me at Mansarovar Park at 5pm wraped')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('Yesterday there was incident of theft near my house')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('A man slapped a girl in the Sadar market')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)

C:\Users\Shreya\anaconda3\lib\site-packages\spacy\displacy\__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)